# LLM Tool Calling + (Simulated) MCP with **Real OpenAI Model** 🧠🛠️

This notebook is a **companion** to the step‑by‑step simulation demo.

Here we plug in a **real OpenAI model** to show how tool calling actually works in code,
while still keeping the MCP side **simulated and simple**.

You will see:

1. A simulated **MCP server** that defines tools (`fake_weather`, `echo`).
2. An **MCP client** that:
   - discovers tools (via a Python list, as if from `list_tools()`),
   - converts them into the OpenAI `tools` schema.
3. A real **OpenAI chat completion call** with `tools=...`.
4. The model returning **tool calls** (tool name + JSON args, no schema).
5. The client executing the tools locally (simulating MCP `call_tool`).
6. A second OpenAI call to turn tool results into a final answer.

> ⚠️ You need an `OPENAI_API_KEY` set in your environment for this notebook to work.


## 0. Setup – Install and Configure OpenAI Client

Run the following cell **once** to install the OpenAI Python client (if not already installed).
Then make sure your `OPENAI_API_KEY` environment variable is set.

In a terminal this typically looks like:

```bash
export OPENAI_API_KEY="sk-..."
```

In Colab, you can set it in `os.environ` as well.


In [ ]:
# If needed, install the OpenAI client library
# !pip install --upgrade openai

import os, json
from typing import Callable
from openai import OpenAI

if not os.getenv("OPENAI_API_KEY"):
    print("⚠️ WARNING: OPENAI_API_KEY is not set. Set it before calling the API.")
else:
    print("✅ OPENAI_API_KEY is set.")

# Choose a tool‑calling capable model.
MODEL = "gpt-4o-mini"  # change if you prefer a different model


## 1. Simulated MCP Server: Tool Definitions

We simulate what an MCP server would expose via `list_tools()`.

Each tool has:
- a **name** (e.g., `fake_weather`),
- a **description**, and
- an `inputSchema` (JSON schema for parameters).


In [ ]:
mcp_tools = [
    {
        "name": "fake_weather",
        "description": "Return a rough current temperature for a city.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "city": {"type": "string", "description": "City name"}
            },
            "required": ["city"],
        },
    },
    {
        "name": "echo",
        "description": "Echo back a message.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "message": {"type": "string", "description": "Message to echo"}
            },
            "required": ["message"],
        },
    },
]

print("Simulated MCP tools (like list_tools() output):")
for t in mcp_tools:
    print(f"- {t['name']}: {t['description']}")


## 2. MCP Client: Convert MCP Tools → OpenAI Tool Schema

The MCP client must convert the MCP tool descriptions into the format expected by
OpenAI's tool‑calling interface.

That is exactly what `mcp_tools_to_openai_tools(...)` does.


In [ ]:
def mcp_tools_to_openai_tools(mcp_tools: list[dict]) -> list[dict]:
    """Convert simulated MCP tools into OpenAI‑style tools list."""
    openai_tools = []
    for tool in mcp_tools:
        openai_tools.append(
            {
                "type": "function",
                "function": {
                    "name": tool["name"],
                    "description": tool["description"],
                    "parameters": tool["inputSchema"],
                },
            }
        )
    return openai_tools

openai_tools = mcp_tools_to_openai_tools(mcp_tools)

print("OpenAI tools schema that will be sent to the model:\n")
print(json.dumps(openai_tools, indent=2))


## 3. First OpenAI Call: Model Decides Whether to Use Tools

Now we perform the **first real OpenAI chat completion** call.

We give the model:
- the user query
- the list of tools

The model may respond with `tool_calls` containing:
- tool name
- JSON arguments

But it will **not** send back any schema.


In [ ]:
client = OpenAI()

user_query = "What is the weather in London right now?"
messages = [{"role": "user", "content": user_query}]

print("Sending first request to OpenAI with tools...\n")
first_response = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools=openai_tools,
)

first_message = first_response.choices[0].message
print("Raw first message from model (repr):\n")
print(first_message)

tool_calls = first_message.tool_calls or []
if tool_calls:
    print("\n✅ Model requested tool calls:")
    for tc in tool_calls:
        print(f"- Tool name: {tc.function.name}")
        print(f"  Arguments JSON: {tc.function.arguments}")
else:
    print("\nℹ️ Model did not request any tool calls.")


## 4. Define Tool Implementations (Simulating MCP `call_tool`)

Now we define the actual Python functions that act like MCP server tools.
The client will use these when the model asks for a tool.


In [ ]:
def tool_fake_weather(city: str) -> str:
    temps = {"London": 18, "Hyderabad": 32, "Bangalore": 26}
    temp = temps.get(city, 25)
    return f"It is roughly {temp}°C in {city} right now."


def tool_echo(message: str) -> str:
    return f"Echo: {message}"


TOOL_REGISTRY: dict[str, Callable[..., str]] = {
    "fake_weather": tool_fake_weather,
    "echo": tool_echo,
}


def execute_tool_call_from_model(tool_call) -> dict:
    """Execute a tool call returned by the model and build a tool message.

    In a real MCP setup, this is where MCP protocol would be used.
    Here we just call local Python functions.
    """
    tool_name = tool_call.function.name
    args = json.loads(tool_call.function.arguments or "{}")
    fn = TOOL_REGISTRY.get(tool_name)
    if fn is None:
        content = f"Error: unknown tool '{tool_name}'"
    else:
        try:
            content = fn(**args)
        except Exception as e:  # noqa: BLE001
            content = f"Error executing tool {tool_name}: {e}"

    tool_message = {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": content,
    }
    return tool_message


tool_messages: list[dict] = []
for tc in tool_calls:
    tm = execute_tool_call_from_model(tc)
    tool_messages.append(tm)

print("Tool messages generated after executing tools:\n")
print(json.dumps(tool_messages, indent=2))


## 5. Second OpenAI Call: Final Answer Using Tool Results

Now we build up the **full conversation history**:

1. User message.
2. Assistant's tool‑call message (from the first response).
3. One or more `role="tool"` messages with the tool outputs.

We send this to OpenAI and ask it to write the final answer.


In [ ]:
full_messages: list[dict] = []
full_messages.append({"role": "user", "content": user_query})

if tool_calls:
    # Add assistant message that contained the tool calls
    full_messages.append({
        "role": "assistant",
        "content": first_message.content or "",
        "tool_calls": [tc.to_dict() for tc in tool_calls],
    })
    # Add tool result messages
    full_messages.extend(tool_messages)
else:
    # If no tool calls, just include the assistant content
    full_messages.append({
        "role": "assistant",
        "content": first_message.content or "",
    })

print("Sending second request to OpenAI with tool results...\n")
second_response = client.chat.completions.create(
    model=MODEL,
    messages=full_messages,
)

final_message = second_response.choices[0].message
print("Final assistant message:\n")
print(final_message)

print("\nAs plain text:\n")
print(final_message.content)


## 6. Summary: What Students Should Take Away

1. **MCP server** defines tools and their JSON schemas.
2. **MCP client** converts those tools into OpenAI's `tools` format.
3. **First OpenAI call**:
   - Model sees the tools and user query.
   - It may return `tool_calls` with **tool name + JSON arguments**.
   - It does **not** send back the schema.
4. **Client executes tools**:
   - In reality, via MCP protocol; here, via local Python functions.
   - Builds `role="tool"` messages with the outputs.
5. **Second OpenAI call**:
   - Model sees the conversation + tool messages.
   - Produces a natural language answer.

👉 The key illusion:

> The LLM does not "speak MCP". The **client** does.
>
> The LLM just
>  - reads tool schemas,
>  - chooses a tool,
>  - fills JSON arguments,
>  - and uses the tool results to answer.
